In [3]:
import Crypto 
import Crypto.Random
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5 
from Crypto.Hash import SHA256
import binascii
import json
from datetime import datetime
import hashlib
import sys

In [4]:
class Transaction:
    def __init__(self, sender, recipient, value):
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = datetime.datetime.now()
    
    def to_dict(self):
        return ({'sender': self.sender, 'recipient': self.recipient, 'value': self.value})
    
    def add_signature(self, signature_):
        self.signature = signature_
    
    def verify_transaction_signature(self):
        if hasattr(self,'signature'):
            public_key = RSA.importKey(binascii.unhexlify(self.sender))
            verifier = PKCS1_v1_5.new(public_key)
            h = SHA256.new(str(self.to_dict()).encode('utf-8'))
            return verifier.verify(h,binascii.unhexlify(self.signature))
        else:
            return False
    
    def to_json(self):
        return json.dumps(self._dict_,sort_keys = False)

In [5]:
class Wallet:
    def __init__(self):
        random = Crypto.Random.new().read
        self._private_key = RSA.generate(1024,random)
        self._public_key = self._private_key.publickey()
        
    def sign_transaction(self,transaction: Transaction):
        signer = PKCS1_v1_5.new(self._private_key)
        h = SHA256.new(str(transaction.to_dict()).encode('utf-8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')
    
    @property
    def identity(self):
        pubkey = binascii.hexlify(self._public_key.exportKey(format="DER"))
        return pubkey.decode('ascii')
    @property
    def private(self):
        privkey = binascii.hexlify(self._private_key.exportKey(format="DER"))
        return privkey.decode('ascii')

In [6]:
Jimmy = Wallet()
Grace = Wallet()

t = Transaction(Grace.identity, Jimmy.identity, 1000)
print('Verifying without a signature result:',t.verify_transaction_signature())

#with Jimmy's signature, the following verification will give out False
signature = Jimmy.sign_transaction(t)
t.add_signature(signature);
print('Verifying using Jimmy\'s signature result:',t.verify_transaction_signature())

#with Grace's signature, the following verification will give out True
signature = Grace.sign_transaction(t)
t.add_signature(signature);
print('Verifying using Grace\'s signature result:',t.verify_transaction_signature())

print('The confirmed transaction signature is:')
print(signature)
print('The length of the signature is :', len(signature))
print("The above program is generated by Suwandi Ryan Loe (student id: 55724681) on", datetime.now())

AttributeError: type object 'datetime.datetime' has no attribute 'datetime'